# 2022 Segunda EDA

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, Lasso, \
LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier

In [2]:
team_stats = pd.read_csv('../../footy_data/segunda_teams_2022.csv')
games = pd.read_csv('../../footy_data/segunda_games_2022.csv')
league = pd.read_csv('../../footy_data/segunda_league_2022.csv')

In [3]:
games = games.loc[games['status'] == 'complete']
games
#Played matches

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1628877600,Aug 13 2021 - 6:00pm,complete,NaN,SD Huesca,SD Eibar,José Antonio López Toca,1,0.0,0.0,...,2.25,2.95,3.50,1.50,2.40,4.25,8.50,2.10,1.71,Estadio El Alcoraz (Huesca)
1,1628884800,Aug 13 2021 - 8:00pm,complete,NaN,Real Zaragoza,UD Ibiza,Iosu Galech Apezteguía,1,0.0,0.0,...,1.78,3.36,5.00,2.50,2.43,4.75,9.50,2.25,1.62,Estadio de la Romareda (Zaragoza)
2,1628953200,Aug 14 2021 - 3:00pm,complete,NaN,Real Sociedad II,Leganés,Víctor García Verdura,1,0.0,0.0,...,3.80,3.00,1.96,1.40,2.40,4.25,8.50,2.10,1.74,Reale Arena (Donostia-San Sebastián)
3,1628962200,Aug 14 2021 - 5:30pm,complete,NaN,Girona FC,SD Amorebieta,David Gálvez Rascón,1,0.0,0.0,...,1.83,3.20,4.80,1.50,2.40,4.40,8.75,2.20,1.65,Estadi Municipal de Montilivi (Girona)
4,1628971200,Aug 14 2021 - 8:00pm,complete,NaN,SD Ponferradina,AD Alcorcón,Javier Iglesias Villanueva,1,0.0,0.0,...,2.55,2.70,3.25,1.71,2.90,5.50,11.00,2.40,1.54,Estadio El Toralín (Ponferrada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,1632052800,Sep 19 2021 - 12:00pm,complete,NaN,SD Huesca,CF Fuenlabrada,Dámaso Arcediano Monescillo,6,2.0,2.0,...,1.95,3.30,4.05,1.45,2.25,3.90,7.75,2.10,1.74,Estadio El Alcoraz (Huesca)
62,1632060000,Sep 19 2021 - 2:00pm,complete,3923.0,SD Ponferradina,Málaga CF,Rafael Sánchez López,6,3.0,0.5,...,2.70,2.95,2.80,1.49,2.30,4.00,7.75,2.00,1.77,Estadio El Toralín (Ponferrada)
63,1632068100,Sep 19 2021 - 4:15pm,complete,NaN,SD Eibar,Sporting Gijón,Francisco José Hernández Maeso,6,0.5,2.0,...,2.10,3.00,3.85,1.56,2.50,4.55,9.25,2.20,1.65,Estadio Municipal de Ipurúa (Eibar)
64,1632068100,Sep 19 2021 - 4:15pm,complete,NaN,CD Tenerife,Mirandés,Luis Mario Milla Alvéndiz,6,2.0,0.5,...,1.57,3.75,6.25,1.45,2.25,4.00,7.75,2.35,1.57,Estadio Heliodoro Rodríguez Lopéz (Santa Cruz ...


In [4]:
#CODE TO FIX XG AND XGA HERE
#average of home_team = common_name[0]xgfor and away_team = common_name[0] xgfor


In [6]:
xg_view = games[['home_team_name', 'away_team_name', 'team_a_xg', 'team_b_xg', 'home_team_goal_count', 'away_team_goal_count']]
xg_view[30:100]

,home_team_name,away_team_name,team_a_xg,team_b_xg,home_team_goal_count,away_team_goal_count
30,Real Sociedad II,CF Fuenlabrada,1.59,0.58,0,0
31,Burgos CF,SD Eibar,1.37,1.27,0,1
32,Real Zaragoza,FC Cartagena,1.96,0.91,0,1
33,Girona FC,Sporting Gijón,2.31,1.73,1,2
34,SD Eibar,Leganés,1.59,1.17,1,1
35,FC Cartagena,Real Sociedad II,1.86,1.53,1,0
36,CD Tenerife,SD Ponferradina,1.83,1.54,2,0
37,Almería,Málaga CF,1.08,2.15,2,0
38,CF Fuenlabrada,CD Lugo,1.71,0.68,1,1
39,Burgos CF,Real Valladolid,1.84,1.21,3,0


In [8]:
team_stats['common_name'][0]

'West Bromwich Albion'

# Goal Analysis

In [10]:
season_goal_total = games['total_goal_count'].sum()
season_1h_goal_total = games['total_goals_at_half_time'].sum()
percent_goals_1h = ((season_1h_goal_total)/(season_goal_total)) * 100

In [11]:
print(f'There were {season_goal_total} goals scored this season')

There were 190 goals scored this season


In [12]:
print(f'There were {season_1h_goal_total} goals scored in the 1H this season, ')

There were 71 goals scored in the 1H this season, 


In [33]:
season_1h_goal_total

71

In [13]:
round(season_1h_goal_total/season_goal_total, 2)
#% of first half goals

0.37

In [15]:
games['total_goals_at_half_time'].value_counts()

1    26
0    26
2    16
3     3
4     1
Name: total_goals_at_half_time, dtype: int64

# Adding more columns

In [16]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go

Let's write a function to help fill xg in our games df

In [17]:
xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]

xga_stats

,common_name,xg_against_avg_overall
0,Stoke City,0.96
1,West Bromwich Albion,0.70
2,Middlesbrough,0.99
3,AFC Bournemouth,0.97
4,Hull City,1.15
5,Swansea City,1.17
6,Queens Park Rangers,1.21
7,Cardiff City,0.83
8,Fulham,0.72
9,Birmingham City,0.81


In [18]:
xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

In [19]:
def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [24]:
fill_stats(team_stats['common_name'][0])
fill_stats(team_stats['common_name'][1])
fill_stats(team_stats['common_name'][2]) 
fill_stats(team_stats['common_name'][3]) 
fill_stats(team_stats['common_name'][4]) 
fill_stats(team_stats['common_name'][5]) 
fill_stats(team_stats['common_name'][6]) 
fill_stats(team_stats['common_name'][7]) 
fill_stats(team_stats['common_name'][8]) 
fill_stats(team_stats['common_name'][9]) 
fill_stats(team_stats['common_name'][10]) 
fill_stats(team_stats['common_name'][11]) 
fill_stats(team_stats['common_name'][12]) 
fill_stats(team_stats['common_name'][13]) 
fill_stats(team_stats['common_name'][14]) 
fill_stats(team_stats['common_name'][15]) 
fill_stats(team_stats['common_name'][16]) 
fill_stats(team_stats['common_name'][17]) 
fill_stats(team_stats['common_name'][18]) 
fill_stats(team_stats['common_name'][19]) 
fill_stats(team_stats['common_name'][20]) 
fill_stats(team_stats['common_name'][21]) 
fill_stats(team_stats['common_name'][22]) 
fill_stats(team_stats['common_name'][23]) 

In [25]:
# fill_stats('Fulham')
# fill_stats('Blackburn Rovers')
# fill_stats('West Bromwich Albion')
# fill_stats('Queens Park Rangers')
# fill_stats('AFC Bournemouth')
# fill_stats('Middlesbrough')
# fill_stats('Barnsley')
# fill_stats('Millwall')
# fill_stats('Sheffield United')
# fill_stats('Reading')
# fill_stats('Cardiff City')
# fill_stats('Nottingham Forest')
# fill_stats('Hull City')
# fill_stats('Blackpool')
# fill_stats('Luton Town')
# fill_stats('Stoke City')
# fill_stats('Swansea City')
# fill_stats('Derby County')
# fill_stats('Huddersfield Town')
# fill_stats('Preston North End')
# fill_stats('Coventry City')
# fill_stats('Peterborough United')
# fill_stats('Birmingham City')
# fill_stats('Bristol City')

#Reading in stats

# Some more EDA

Let's write a function to help fill xg in our games df

In [26]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go



xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]

xga_stats.head()

xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

fill_stats('Fulham')
fill_stats('Blackburn Rovers')
fill_stats('West Bromwich Albion')
fill_stats('Queens Park Rangers')
fill_stats('AFC Bournemouth')
fill_stats('Middlesbrough')
fill_stats('Barnsley')
fill_stats('Millwall')
fill_stats('Sheffield United')
fill_stats('Reading')
fill_stats('Cardiff City')
fill_stats('Nottingham Forest')
fill_stats('Hull City')
fill_stats('Blackpool')
fill_stats('Luton Town')
fill_stats('Stoke City')
fill_stats('Swansea City')
fill_stats('Derby County')
fill_stats('Huddersfield Town')
fill_stats('Preston North End')
fill_stats('Coventry City')
fill_stats('Peterborough United')
fill_stats('Birmingham City')
fill_stats('Bristol City')

#Reading in stats

# Some more EDA

# Over/Under 2.5?

In [27]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5

In [28]:
games['over/under'].value_counts()

1    38
0    34
Name: over/under, dtype: int64

# xG Over/Under 2.5? 

In [20]:
games['xg_over/under'] = np.where(games['team_a_xg'] + games['team_b_xg'] > 2.99, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5
games['xg_over/under_1.99'] = np.where(games['team_a_xg'] + games['team_b_xg'] > 1.99, 1, 0)
# Adding column 0 represents under 1.5, 1 represents over 1.5

In [21]:
games['xg_over/under'].value_counts()

0    37
1    23
Name: xg_over/under, dtype: int64

In [22]:
games['xg_over/under_1.99'].value_counts()

1    59
0     1
Name: xg_over/under_1.99, dtype: int64

# First Week

In [23]:
games['over/under_1.5'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 1.5, 1, 0)
# Adding column 0 represents under 1.5, 1 represents over 1.5

In [23]:
first_matchweek = games[0:12]

In [24]:
first_matchweek['over/under'].value_counts()

0    7
1    5
Name: over/under, dtype: int64

In [25]:
first_matchweek['over/under_1.5'].value_counts()

1    9
0    3
Name: over/under_1.5, dtype: int64

In [26]:
first_matchweek['xg_over/under'].value_counts()

0    7
1    5
Name: xg_over/under, dtype: int64

In [27]:
first_matchweek['xg_over/under_1.99'].value_counts()

1    11
0     1
Name: xg_over/under_1.99, dtype: int64

In [28]:
games['home_team_goal_count_half_time'].sum()

7

In [29]:
games['away_team_goal_count_half_time'].sum()

4